<a href="https://colab.research.google.com/github/hamish-haggerty/AI-hacking/blob/master/SSL/cancer_random_initial_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cancer_random_initial_weights

> Ok, purpose of this notebook is to do the following:

    1) Fine tune models across two conditions: pretrain=True and pretrain=False. We show that ensembling across conditions is better than within. Also, importantly, save all the results: Then we can re-use e.g. the BT trained nets (but not fine tuned) in other ensemble experiments. Cool.
    2) Check list: copy paste `main_train` in temporarily, as well as `predict` (average in probability space rather than score space for stability). ALSO importantly try and ensure reproducibility by setting the appropriate flags (seed_everything etc)


Speaking a bit more generally, the plan of attack for the rest of today is the following: Work towards training final models for this notebook and the supervised vs self-supervised intitial weights. Tinker with the decorrelation idea concurrently, but we are not going to spend too long on that. It either works or it doesn't, basically. Then perhaps by tomorrow we will be training our final models on the decorrelation idea as well. So, by THURSDAY we can have 100% (basically) of our results, and writing the paper can continue in earnest. Getting the results is the hard part, once we have that we can do shit loads of writing in 1 week alone (but we don't want to leave it too late). 

You can do it, but you have to work well from this point on. The key is consistency, not intensity. So what, you procrastinated for an hour. So long as you then work well for a few hours. Cool.

In [1]:
#| default_exp cancer_random_initial_weights

In [2]:
#| hide
import os
from google.colab import drive

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():

    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')

    os.chdir('cancer-proj')
    
    os.system('pip install .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

Mounted at /content/drive


In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
import os
os.system('pip install .')
os.system('pip install git+https://github.com/hamish-haggerty/base_rbt.git')

0

In [5]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *
from cancer_proj.cancer_maintrain import *
from cancer_proj.cancer_maintrain_ensemble import *


## Load the data

In [6]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [7]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

## Optionally, display:

In [8]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [9]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [10]:
#| export

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

## Note: we don't actually want to freeze the resnet when pretraining in this notebook, since the initial weights are random. So we just need to make sure encoder_fine_tune isn't doing that

## Test: make sure that it freezes the appropriate part of model:

Make sure resnet frozen, linear head unfrozen:

In [11]:
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
model = LM(encoder)
test_eq(len(my_splitter(model)),2)
test_eq(len(my_splitter_bt(bt_model)),2)

learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.freeze()
print('body should be frozen, (sans batchnorm) linear head unfrozen')
learn.summary()

Downloading: "https://github.com/facebookresearch/barlowtwins/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/barlowtwins/ep1000_bs2048_lrw0.2_lrb0.0048_lambd0.0051/resnet50.pth" to /root/.cache/torch/hub/checkpoints/resnet50.pth


  0%|          | 0.00/90.0M [00:00<?, ?B/s]

body should be frozen, (sans batchnorm) linear head unfrozen


LM (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                               

## We can use this to check the  bt-model is of the appropriate type: resnet + projector

In [12]:
#| hide

#test : manual. BT

learn = Learner(dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
learn.freeze()
print('body should be frozen, (sans batchnorm) projector unfrozen')
learn.summary()

body should be frozen, (sans batchnorm) projector unfrozen


BarlowTwinsModel (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                 

## Reproducibility

In [13]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Setup directory for saving

In [14]:
import os

directory = '/content/drive/My Drive/random_initial_weights'

if not os.path.exists(directory):
    os.makedirs(directory)

In [15]:
def run_main_train(pretrain,initial_weights='no_pretrain',num_epochs=300,freeze_num_epochs=1,freeze_numfit=6,numfit=75,num=5):
    "run main_train num times."


    main_dict = {}
    for i in range(num):

        main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
                aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
                initial_weights=initial_weights,pretrain=pretrain,
                num_epochs=num_epochs,numfit=numfit,freeze_num_epochs=freeze_num_epochs,freeze_numfit=freeze_numfit,
                print_report=True,
                tune_model_path = directory + f'/pretrain={pretrain}_{i}.pth'
                        )
        
        #This is so we can re-use the models we train here elsewhere...
        if pretrain==True:
            main.train_encoder()
            bt_model_path = directory+f'/bt_model_{i}'
            torch.save(main.bt_model.state_dict(), bt_model_path) #i.e. bt_model_{0,1,2 etc}
        else:
            bt_model_path=None

        metrics = main.fine_tune()
        
        metrics['bt_model_path'] =  bt_model_path #only interesting under bt pretraining...

        main_dict[i] = metrics

    return main_dict
        

In [16]:
lst = [False,True] #pretrain or not
results={False:None,True:None} 

for pretrain in lst:
    main_dict = run_main_train(pretrain=pretrain,num=6)
    results[pretrain] = main_dict #main_dict has result of running main num=3 times.

save_dict_to_gdrive(d=results,directory=directory,filename='results')

pretrain was False, and about to fit_one_cycle


epoch,train_loss,valid_loss,time
0,2.333055,None,00:07
1,2.308322,None,00:06
2,2.269488,None,00:06
3,2.237971,None,00:06
4,2.213191,None,00:06
5,2.196490,None,00:06
6,2.178480,None,00:06
7,2.167102,None,00:06
8,2.151303,None,00:06
9,2.134321,None,00:06


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


                            precision    recall  f1-score   support

         actinic keratosis       0.35      0.30      0.32        20
      basal cell carcinoma       0.18      0.20      0.19        20
            dermatofibroma       0.30      0.37      0.33        19
                  melanoma       0.29      0.35      0.32        20
                     nevus       0.57      0.65      0.60        20
pigmented benign keratosis       0.43      0.50      0.47        20
      seborrheic keratosis       0.29      0.13      0.18        15
   squamous cell carcinoma       0.33      0.30      0.32        20
           vascular lesion       0.88      0.75      0.81        20

                  accuracy                           0.40       174
                 macro avg       0.40      0.39      0.39       174
              weighted avg       0.41      0.40      0.40       174

pretrain was False, and about to fit_one_cycle


epoch,train_loss,valid_loss,time
0,2.286158,None,00:06
1,2.275197,None,00:06
2,2.263676,None,00:06
3,2.239666,None,00:06
4,2.220148,None,00:07
5,2.200195,None,00:06
6,2.181748,None,00:06
7,2.164324,None,00:06
8,2.152082,None,00:06
9,2.137261,None,00:06


...

/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


pretrain was True, and about to linear_fine_tune


epoch,train_loss,valid_loss,time
0,2.513163,None,00:08
1,2.407678,None,00:08
2,2.347336,None,00:08
3,2.280573,None,00:08
4,2.241857,None,00:08
5,2.228123,None,00:07


epoch,train_loss,valid_loss,time
0,1.990890,None,00:07
1,1.919512,None,00:08
2,1.858858,None,00:08
3,1.806976,None,00:08
4,1.758358,None,00:08
5,1.700629,None,00:08
6,1.643313,None,00:08
7,1.587973,None,00:08
8,1.544989,None,00:07
9,1.507112,None,00:08


                            precision    recall  f1-score   support

         actinic keratosis       0.40      0.50      0.44        20
      basal cell carcinoma       0.38      0.30      0.33        20
            dermatofibroma       0.60      0.47      0.53        19
                  melanoma       0.37      0.50      0.43        20
                     nevus       0.59      0.50      0.54        20
pigmented benign keratosis       0.45      0.50      0.48        20
      seborrheic keratosis       0.50      0.33      0.40        15
   squamous cell carcinoma       0.50      0.60      0.55        20
           vascular lesion       0.94      0.85      0.89        20

                  accuracy                           0.51       174
                 macro avg       0.53      0.51      0.51       174
              weighted avg       0.53      0.51      0.51       174



In [17]:
# results = load_dict_from_gdrive(directory=directory,filename='results')
# xval = results[False][0]['xval']
# yval = results[False][0]['yval']
# lst = [False,True] #pretrain or not


In [18]:
# for i in range(2):
#     print(results[True][i]['classif_model_path'])
#     print(results[True][i]['bt_model_path'])

In [19]:
# #How to load
# results = load_dict_from_gdrive(directory=directory,filename='results')
# path = results[False][0]['classif_model_path']
# # #init model
# _,encoder = create_model(which_model='no_pretrain',ps=8192,device=device)
# model = LM(encoder)
# model.load_state_dict(torch.load(path, map_location='cuda:0'))

In [20]:
#How to load
#results = load_dict_from_gdrive(directory=directory,filename='results')
# path = results[False][0]['classif_model_path']
# # #init model
# _,encoder = create_model(which_model='no_pretrain',ps=8192,device=device)
# model = LM(encoder)
# model.load_state_dict(torch.load(path, map_location='cuda:0'))

#Print out result of each run:

In [27]:
from statistics import mean,stdev
print('random initial weights: ')
for pretrain in lst:

    _results = [results[pretrain][i]['acc'] for i in range(len(results[pretrain]))]

    print(f"\npretrain={pretrain}, results are: {_results}")
    print(f"mean is: {mean(_results)}")
    print(f"std is: {stdev(_results)}\n")


random initial weights: 

pretrain=False, results are: [0.40229883790016174, 0.44252872467041016, 0.44252872467041016, 0.43103447556495667, 0.4597701132297516, 0.4482758641242981]
mean is: 0.4377394566933314
std is: 0.019714127507268586


pretrain=True, results are: [0.5344827771186829, 0.5114942789077759, 0.540229856967926, 0.5114942789077759, 0.5114942789077759, 0.5114942789077759]
mean is: 0.5201149582862854
std is: 0.013478188444753993



In [28]:
from itertools import combinations

print('Ensembling within type of initial weights')

def within_ensemble_results(results,key,param='pretrain'):

    print(f'Results for {param}={key}')

    _results = list(results[key].values())
    _results = list(combinations(_results,2)) #all pairs of results. So for num=3, will be 3
    lst=[]
    for v in _results:

        print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
        print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
        _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
        print(f'Acc of ensemble is:{acc}\n')
        lst.append(acc)

    print(f"mean is: {mean(lst)}")
    print(f"std is: {stdev(lst)}\n")

Ensembling within type of initial weights


In [29]:
within_ensemble_results(results,key=False)

Results for pretrain=False

Acc of first guy in ensemble is: 0.40229883790016174
Acc of second guy in ensemble is: 0.44252872467041016
Acc of ensemble is:0.4655172526836395


Acc of first guy in ensemble is: 0.40229883790016174
Acc of second guy in ensemble is: 0.44252872467041016
Acc of ensemble is:0.4597701132297516


Acc of first guy in ensemble is: 0.40229883790016174
Acc of second guy in ensemble is: 0.43103447556495667
Acc of ensemble is:0.41379308700561523


Acc of first guy in ensemble is: 0.40229883790016174
Acc of second guy in ensemble is: 0.4597701132297516
Acc of ensemble is:0.4367816150188446


Acc of first guy in ensemble is: 0.40229883790016174
Acc of second guy in ensemble is: 0.4482758641242981
Acc of ensemble is:0.44252872467041016


Acc of first guy in ensemble is: 0.44252872467041016
Acc of second guy in ensemble is: 0.44252872467041016
Acc of ensemble is:0.4655172526836395


Acc of first guy in ensemble is: 0.44252872467041016
Acc of second guy in ensemble is: 0.4

In [30]:
within_ensemble_results(results,key=True)

Results for pretrain=True

Acc of first guy in ensemble is: 0.5344827771186829
Acc of second guy in ensemble is: 0.5114942789077759
Acc of ensemble is:0.540229856967926


Acc of first guy in ensemble is: 0.5344827771186829
Acc of second guy in ensemble is: 0.540229856967926
Acc of ensemble is:0.545976996421814


Acc of first guy in ensemble is: 0.5344827771186829
Acc of second guy in ensemble is: 0.5114942789077759
Acc of ensemble is:0.545976996421814


Acc of first guy in ensemble is: 0.5344827771186829
Acc of second guy in ensemble is: 0.5114942789077759
Acc of ensemble is:0.568965494632721


Acc of first guy in ensemble is: 0.5344827771186829
Acc of second guy in ensemble is: 0.5114942789077759
Acc of ensemble is:0.5517241358757019


Acc of first guy in ensemble is: 0.5114942789077759
Acc of second guy in ensemble is: 0.540229856967926
Acc of ensemble is:0.5344827771186829


Acc of first guy in ensemble is: 0.5114942789077759
Acc of second guy in ensemble is: 0.5114942789077759
Acc 

In [25]:
import itertools
print('Ensembling between')

def between_ensemble_results(results,key1,key2,param='pretrain'):

    #could use zip here also for uniformity

    _results = list(itertools.product(results[key1].values(),results[key2].values()))

    for v in _results:

        print(f"\nAcc of first guy in ensemble ({key1}) is: {v[0]['acc']}")
        print(f"Acc of second guy in ensemble ({key2}) is: {v[1]['acc']}")
        _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
        print(f'Acc of ensemble is:{acc}\n')


 

Ensembling between


Between ensemble doesn't matter as much here: i.e. between is between pretrain=True and pretrain=False

In [26]:
between_ensemble_results(results,key1=True,key2=False)


Acc of first guy in ensemble (True) is: 0.5344827771186829
Acc of second guy in ensemble (False) is: 0.40229883790016174
Acc of ensemble is:0.545976996421814


Acc of first guy in ensemble (True) is: 0.5344827771186829
Acc of second guy in ensemble (False) is: 0.44252872467041016
Acc of ensemble is:0.545976996421814


Acc of first guy in ensemble (True) is: 0.5344827771186829
Acc of second guy in ensemble (False) is: 0.44252872467041016
Acc of ensemble is:0.545976996421814


Acc of first guy in ensemble (True) is: 0.5344827771186829
Acc of second guy in ensemble (False) is: 0.43103447556495667
Acc of ensemble is:0.522988498210907


Acc of first guy in ensemble (True) is: 0.5344827771186829
Acc of second guy in ensemble (False) is: 0.4597701132297516
Acc of ensemble is:0.540229856967926


Acc of first guy in ensemble (True) is: 0.5344827771186829
Acc of second guy in ensemble (False) is: 0.4482758641242981
Acc of ensemble is:0.517241358757019


Acc of first guy in ensemble (True) is: 0